### This notebook provides a rough idea of how to combine the similar items in the transactions into unified value. This is to be done in order to reduce the item size.

#### Note: So far, i have managed to reduce 5000 different items to approx 3300 items which is considerable difference when it comes to computation costs. This number can be increased / decreased by modifying the hyperparameters like similarity threshold and jaccard token size.



In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#reading the data
df = pd.read_excel('online_merged.xlsx', sheet_name='merged')

In [3]:
# Stripping white spaces from the item description column
df['Description'] = df['Description'].str.strip()

In [4]:
# Defining functions for finding Jaccard similarities

def Intersection(lst1, lst2): 
    return list(set(lst1).intersection(lst2))

def Union(lst1, lst2): 
    final_list = list(set(lst1) | set(lst2)) 
    return final_list 

def tokens(word,token_size = 3):
    return [word[i:i+token_size] for i in range(0,len(word)-token_size + 1)]

def find_jaccard(word1, word2, token_size = 3):
    tokens1 = tokens(word1, token_size)
    tokens2 = tokens(word2, token_size)
    return (len(Intersection(tokens1, tokens2))/len(Union(tokens1, tokens2))) 

In [5]:
# Unique item description list

unq_desc = df.Description.unique()

In [6]:
# Sample list of items to show the working
list1 = ['CHOCOLATE BOX RIBBONS',
'DINOSAUR LUNCH BOX WITH CUTLERY',
'DINOSAUR LUNCHBOX WITH CUTLERY',
'EASTER BUNNY WITH BASKET ON BACK',
'EASTER BUNNY WREATH',
'ELEPHANT TOY WITH BLUE T-SHIRT',
'HEART BUTTONS JEWELLERY BOX',
'LUNCH BOX WITH CUTLERY FAIRY CAKES',
'LUNCH BOX WITH CUTLERY RETROSPOT',
'LUNCHBOX WITH CUTLERY FAIRY CAKES',
'LUNCHBOX WITH CUTLERY RETROSPOT',
'NEW BAROQUE IVORY CUSHION COVER',
'SKULL LUNCH BOX WITH CUTLERY',
'SKULL LUNCHBOX WITH CUTLERY',
'STRAWBERRY LUNCH BOX WITH CUTLERY',
'STRAWBERRY LUNCHBOX WITH CUTLERY',
'UTILTY CABINET WITH HOOKS']
list1

['CHOCOLATE BOX RIBBONS',
 'DINOSAUR LUNCH BOX WITH CUTLERY',
 'DINOSAUR LUNCHBOX WITH CUTLERY',
 'EASTER BUNNY WITH BASKET ON BACK',
 'EASTER BUNNY WREATH',
 'ELEPHANT TOY WITH BLUE T-SHIRT',
 'HEART BUTTONS JEWELLERY BOX',
 'LUNCH BOX WITH CUTLERY FAIRY CAKES',
 'LUNCH BOX WITH CUTLERY RETROSPOT',
 'LUNCHBOX WITH CUTLERY FAIRY CAKES',
 'LUNCHBOX WITH CUTLERY RETROSPOT',
 'NEW BAROQUE IVORY CUSHION COVER',
 'SKULL LUNCH BOX WITH CUTLERY',
 'SKULL LUNCHBOX WITH CUTLERY',
 'STRAWBERRY LUNCH BOX WITH CUTLERY',
 'STRAWBERRY LUNCHBOX WITH CUTLERY',
 'UTILTY CABINET WITH HOOKS']

In [7]:
# Grouping the similar items based on unique item description list to create a lookup result

from difflib import SequenceMatcher as sm

combined_list = list1 # Use unq_desc to get complete list of combinations from txns.
result = list()
result_group = list()
usedElements = list()
skip = False

for firstName in combined_list:
    skip = False

    for x in usedElements:
        if x == firstName:
            skip = True
    if skip == True:
        continue

    for secondName in combined_list:

        #if(sm(None, firstName, secondName).ratio() >= 0.70):            
        if(find_jaccard(firstName, secondName) >= 0.50):            

            result_group.append(secondName)
            usedElements.append(secondName)
        else:
            pass

    result.append(result_group[:])
    del result_group[:]

for value in result:
    print(value)

['CHOCOLATE BOX RIBBONS']
['DINOSAUR LUNCH BOX WITH CUTLERY', 'DINOSAUR LUNCHBOX WITH CUTLERY', 'LUNCH BOX WITH CUTLERY FAIRY CAKES', 'LUNCH BOX WITH CUTLERY RETROSPOT', 'SKULL LUNCH BOX WITH CUTLERY', 'SKULL LUNCHBOX WITH CUTLERY', 'STRAWBERRY LUNCH BOX WITH CUTLERY']
['EASTER BUNNY WITH BASKET ON BACK']
['EASTER BUNNY WREATH']
['ELEPHANT TOY WITH BLUE T-SHIRT']
['HEART BUTTONS JEWELLERY BOX']
['LUNCH BOX WITH CUTLERY FAIRY CAKES', 'LUNCHBOX WITH CUTLERY FAIRY CAKES', 'LUNCHBOX WITH CUTLERY RETROSPOT', 'SKULL LUNCHBOX WITH CUTLERY', 'STRAWBERRY LUNCHBOX WITH CUTLERY']
['NEW BAROQUE IVORY CUSHION COVER']
['UTILTY CABINET WITH HOOKS']


In [8]:
df1 = df
df1.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,493414,21844,RETRO SPOT MUG,36,2010-01-04 10:28:00,2.55,14590,United Kingdom
1,493414,21533,RETRO SPOT LARGE MILK JUG,12,2010-01-04 10:28:00,4.25,14590,United Kingdom
2,493414,37508,NEW ENGLAND CERAMIC CAKE SERVER,2,2010-01-04 10:28:00,2.55,14590,United Kingdom
3,493414,35001G,HAND OPEN SHAPE GOLD,2,2010-01-04 10:28:00,4.25,14590,United Kingdom
4,493414,21527,RETRO SPOT TRADITIONAL TEAPOT,12,2010-01-04 10:28:00,6.95,14590,United Kingdom


In [9]:
df1_price = df1[(df1[['Price','Quantity']] > 0).all(axis=1)]
df1.describe()

,Invoice,Quantity,Price,Customer ID
count,770692.000000,770692.000000,770692.000000,770692.000000
mean,539271.351953,13.286099,2.993243,15327.973197
std,25509.968171,146.100973,11.160934,1696.651545
min,493414.000000,1.000000,0.001000,12346.000000
25%,517964.000000,2.000000,1.250000,13975.000000
50%,537868.000000,5.000000,1.950000,15260.000000
75%,562460.000000,12.000000,3.750000,16800.000000
max,581587.000000,80995.000000,8142.750000,18287.000000


In [10]:
# Creating a sample transaction dataframe from origional DF for testing purposes.

#np.random.seed(123)
#sample_desc = np.random.choice(unq_desc, size = 7)

sample_txn = df1[df1['Description'].isin(list1)]

sample_txn = sample_txn.sample(n = 100)

len(sample_txn.Invoice.unique())

sample_txn.groupby(['StockCode']).count()

sample_txn['TotalPrice'] = sample_txn['Price']*sample_txn['Quantity']

sample_txn

#basket = (sample_txn.groupby(['Invoice', 'Description'])['TotalPrice'] 
#          .sum().unstack().reset_index().fillna(0) 
#          .set_index('Invoice')) 
#basket

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
36399,498395,21559,STRAWBERRY LUNCHBOX WITH CUTLERY,240,2010-02-18 15:13:00,2.10,14646,Netherlands,504.00
323728,532636,21561,DINOSAUR LUNCH BOX WITH CUTLERY,6,2010-11-12 15:46:00,2.55,12635,Germany,15.30
353277,535662,21559,STRAWBERRY LUNCH BOX WITH CUTLERY,1,2010-11-28 11:37:00,2.55,16782,United Kingdom,2.55
437977,544824,22352,LUNCH BOX WITH CUTLERY RETROSPOT,6,2011-02-24 09:32:00,2.55,17139,United Kingdom,15.30
46346,499772,21559,STRAWBERRY LUNCHBOX WITH CUTLERY,2,2010-03-02 13:36:00,2.55,16960,United Kingdom,5.10
...,...,...,...,...,...,...,...,...,...
450814,546392,21561,DINOSAUR LUNCH BOX WITH CUTLERY,1,2011-03-11 14:15:00,2.55,14587,United Kingdom,2.55
119340,508654,22353,LUNCH BOX WITH CUTLERY FAIRY CAKES,6,2010-05-17 15:54:00,2.55,15296,United Kingdom,15.30
160772,513809,21559,STRAWBERRY LUNCH BOX WITH CUTLERY,6,2010-06-29 09:28:00,2.55,12682,France,15.30
585815,563360,21558,SKULL LUNCH BOX WITH CUTLERY,4,2011-08-15 15:06:00,2.55,15023,United Kingdom,10.20


In [11]:
# Checking the groupped values before unifying the descriptions

sums = sample_txn.groupby(['Description']).sum()
sums

,Invoice,Quantity,Price,Customer ID,TotalPrice
Description,,,,,
CHOCOLATE BOX RIBBONS,6325301,81,15.00,186537,101.25
DINOSAUR LUNCH BOX WITH CUTLERY,6966806,45,33.15,204946,114.75
EASTER BUNNY WITH BASKET ON BACK,547896,2,0.65,17841,1.30
EASTER BUNNY WREATH,1094135,3,9.90,32317,14.85
ELEPHANT TOY WITH BLUE T-SHIRT,997367,3,7.50,27870,11.25
HEART BUTTONS JEWELLERY BOX,513625,1,7.65,14505,7.65
LUNCH BOX WITH CUTLERY FAIRY CAKES,4616419,48,22.95,135733,122.40
LUNCH BOX WITH CUTLERY RETROSPOT,10138611,100,48.45,282839,255.00
LUNCHBOX WITH CUTLERY FAIRY CAKES,499302,6,2.55,18223,15.30


In [12]:
#unifying the item description using lookup from results

for index, row in sample_txn.iterrows():
    
    for desc_list in result:
            #if col in 
        if(row['Description'] in desc_list):
            print('before conversion:')
            print(row['Description'], ',', desc_list[0])
            sample_txn.at[index,'Description'] =  desc_list[0]
            print('After conversion:')
            print(row['Description'], ',', desc_list[0])
            print('-------------------------------------------------------------------------')
            
            break
            #print('-------------------------------------------------------------------------')
            #print(round(row.sum(),2))
#col_list_full    

before conversion:
STRAWBERRY LUNCHBOX WITH CUTLERY , LUNCH BOX WITH CUTLERY FAIRY CAKES
After conversion:
STRAWBERRY LUNCHBOX WITH CUTLERY , LUNCH BOX WITH CUTLERY FAIRY CAKES
-------------------------------------------------------------------------
before conversion:
DINOSAUR LUNCH BOX WITH CUTLERY , DINOSAUR LUNCH BOX WITH CUTLERY
After conversion:
DINOSAUR LUNCH BOX WITH CUTLERY , DINOSAUR LUNCH BOX WITH CUTLERY
-------------------------------------------------------------------------
before conversion:
STRAWBERRY LUNCH BOX WITH CUTLERY , DINOSAUR LUNCH BOX WITH CUTLERY
After conversion:
STRAWBERRY LUNCH BOX WITH CUTLERY , DINOSAUR LUNCH BOX WITH CUTLERY
-------------------------------------------------------------------------
before conversion:
LUNCH BOX WITH CUTLERY RETROSPOT , DINOSAUR LUNCH BOX WITH CUTLERY
After conversion:
LUNCH BOX WITH CUTLERY RETROSPOT , DINOSAUR LUNCH BOX WITH CUTLERY
-------------------------------------------------------------------------
before convers

In [13]:
# Checking the groupped values after unifying the descriptions

sums = sample_txn.groupby(['Description']).sum()
sums

,Invoice,Quantity,Price,Customer ID,TotalPrice
Description,,,,,
CHOCOLATE BOX RIBBONS,6325301,81,15.00,186537,101.25
DINOSAUR LUNCH BOX WITH CUTLERY,40122355,526,189.90,1108444,1260.30
EASTER BUNNY WITH BASKET ON BACK,547896,2,0.65,17841,1.30
EASTER BUNNY WREATH,1094135,3,9.90,32317,14.85
ELEPHANT TOY WITH BLUE T-SHIRT,997367,3,7.50,27870,11.25
HEART BUTTONS JEWELLERY BOX,513625,1,7.65,14505,7.65
LUNCH BOX WITH CUTLERY FAIRY CAKES,3496538,260,17.40,112079,555.00


In [14]:
sums['Description']

KeyError: 'Description'